In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

## S.3 데이터 변환

- 데이터로부터 좀 더 의미있는 신호를 읽기 위해서는 예측, 분류, 군집화, 추천 등 기계학습 필요


- **ETL** (Extract, Transform, Load)은 소스에서 필요한 데이터를 추출, 변환하고, 다른 타겟으로 로딩하는 것으로 말함
  - **추출**은 원천에서 데이터를 가져오는 것, spark에서는 예를 들어 csv, NoSQL 등에서 데이터를 읽어서 RDD, DataFrame을 생성하는 작업을 말한다.
  - **변환**은 분석 가능한 형식으로 변환하는 것으로, 결측 값이나 이상값을 제거하거나 map() 함수를 사용하거나 데이터타입 변환을 하는 등의 작업을 한다.
  - **로딩**은 변환한 데이터를 저장하여 놓는 것으로 Spark에서는 RDD, DataFrame의 형식으로 만들어 놓는다. 지도학습 Supervised Learning을 하려면, **DataFrame은 label, features 컬럼을, RDD는 label과 features를 가지고 있는 Labeled Point**로 구성해야 한다.

In [67]:
import iplantuml

In [ ]:
%%plantuml --jar
@startuml
database source
rectangle transform
source -right-> transform: extract
database target
transform -right-> target: load
@enduml

ml 라이브러리가 새로운 기능이 추가되고 있음. mllib는 유지보수만.

패키지|설명|데이터 타입 예
:---:|:---:|:---:
mllib|RDD API를 제공|`pyspark.mllib.linalg.Vector` 또는 `pyspark.mllib.linalg.Matrix`
ml|DataFrame API를 제공|`pyspark.ml.linalg.Vector` 또는 `pyspark.ml.linalg.Matrix`

## S.4 RDD 변환

구분|설명
:---:|:---:
Vector|numpy vector와 같은 기능을 한다. **dense**와 **sparse** vector로 구분한다.
Labeled Point|분류를 의미하는 클래스 또는 **label**과 속성 **features** 이 묶인 구조로서, 지도학습 supervised learning을 할 경우 사용된다.
Matrix|numpy matrix와 같은 특징을 가진다.

### S.4.1 vectors
행렬 Vector는 dense와 sparse로 구분할 수 있다.

- **dense vector**는 빈 값이 별로 없이 모든 행렬이 값을 가지고 있다.
- **sparse vector**는 빈 값이 많아서, 값이 있는 경우 그 값이 있는 인덱스로 표현해 배열을 축약하게 된다.

#### Dense Vectors 밀집벡터
- 벡터는 일련의 수로 구성이 되고, 행벡터 또는 열벡터가 될 수 있다. 

- 채워지는 값이 대부분 0이면 희소벡터 Sparse Vectors로 만들어 질 수 있다.

In [3]:
import numpy as np

In [4]:
# numpy array를 사용하면 만들어지는 것이 dense vector
dv = np.array([1.0, 2.1, 3]) #dv: dense vector numpy로 만들기

Spark에서는 Vectors 명령어로 벡터를 만들 수 있다.

단 **RDD는 mllib 모듈**을 사용해서 vectors를 만들고, 반면에 **DataFrame은 ml**를 사용한다는 점에 유의하자.

In [5]:
## pyspark.mllib 모듈을 이용하여 Vectors를 생성
from pyspark.mllib.linalg import Vectors

dv1mllib=Vectors.dense([1.0, 2.1, 3]) 

In [6]:
print ("Dense vector: {}\nType: {}".format(dv1mllib, type(dv1mllib)))

Dense vector: [1.0,2.1,3.0]
Type: <class 'pyspark.mllib.linalg.DenseVector'>


In [7]:
# pyspark.ml 모듈을 이용하여 Vectors를 생성
from pyspark.ml.linalg import Vectors

dv1ml=Vectors.dense([1.0, 2.1, 3])

In [8]:
print ("ml의 dense vector: {}".format(dv1ml))

ml의 dense vector: [1.0,2.1,3.0]


In [9]:
# numpy array와 같은 특징 : 인덱스로 값 읽기, 반복문 사용 가능
for e in dv1mllib:
    print (e, end = " ")

1.0 2.1 3.0 

In [10]:
#  product, dot, norm과 같은 벡터 연산을 할 수도 있다
dv1mllib.dot(dv1mllib)

14.41

In [11]:
np.dot(dv,dv)

14.41

In [12]:
# 더하기, 빼기, 곱하기, 나누기 연산도 가능
# 곱하기 연산을 해보자. dot와 달리 항목별로 실행됨.
dv1mllib*dv1mllib

DenseVector([1.0, 4.41, 9.0])

#### Sparse Vectors 희소행렬
- 행렬에는 0 값이 많이 존재하기 때문에, 0값이 아닌 **NZ** Nonzero만 저장하면 훨씬 효율적
- **sparse는 실제 값이 없는 요소**, **'0'을 제거**하여 만든 vector이다.
  - Spark에서 type field (1 바이트 길이)를 통해 식별한다 (0: sparse, 1: dense)
  

예를 들어, 다음은 1차원 dense vector이다.

> [160, 69, 0, 0, 24]

sparse vectors는 값 중에 **0이 포함된 경우 이를 생략**하고, 값이 있는 요소 Nonzero만 남기게 된다.  
3은 컬럼 갯수, 0, 1, 4는 값이 있는 컬럼, [160.0, 69.0, 24.0]는 실제 값을 의미한다.

In [13]:
sv1 = Vectors.sparse(5,[0,1,4],[160.0,69.0,24.0])

# 요소가 5개인데, 0,1,4번째에 값이 있고, 그 값은 순서대로 ---이다.
# 장소 효율적으로 사용 가능

In [14]:
# mllib 또는 ml인지 확인
type(sv1)

pyspark.ml.linalg.SparseVector

`toArray()` 함수를 사용하면 sparse에서 dense로 벡터를 변환할 수 있다

In [15]:
sv1.toArray()

array([160.,  69.,   0.,   0.,  24.])

### S.4.2 Matrix

#### Dense Matrix

In [16]:
from pyspark.mllib.linalg import Matrices

Matrices.dense(3, 2, [1,2,3,4,5,6])

DenseMatrix(3, 2, [1.0, 2.0, 3.0, 4.0, 5.0, 6.0], False)

#### Sparse Vectors
>[1 0 2]  
[0 0 3]  
[4 5 6]

위와 같은 2차원 dense vectors를 sparse vectors의 배열 방식으로 표현해보자. 우선 다음과 같이 행, 열, 값 vector를 만든다.

>행 | 0 | 0 | 1 | 2 | 2 | 2   `# 0번째에 값 2번 등장 -> 0 2개....`  
열 | 0 | 2 | 2 | 0 | 1 | 2   `몇번째 행에 값 나오는가를 세로로 생각`  
값 | 1 | 2 | 3 | 4 | 5 | 6

행을 보면 0번째에 '1','2' 1번째에 '3', 2번째에 '4','5','6'이므로 0,0,1,2,2,2  
열을 보면 0번째에 '1', 2번째 '2','3', 0번째 '4', 1번째 '5', 2번째 '6'이므로 0,2,2,0,1,2

행, 열, 데이터를 한 쌍으로 읽으면 된다. 즉 행 0, 열 0의 위치에 1, 행 0, 열 2의 위치에 2. 이런 식으로 6개의 데이터가 있다.

In [17]:
import numpy as np

row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])

In [18]:
# scipy의 sparse vector로 표현해보기
import scipy.sparse as sps

mtx = sps.csc_matrix((data, (row, col)), shape=(3, 3))

In [19]:
print (mtx.todense())

[[1 0 2]
 [0 0 3]
 [4 5 6]]


#### Sparse Vectors의 CSR (Compressed Sparse Row) 또는 Yale Format
다음 5개의 값으로 표현된다.

1. 첫째 행 개수 (int)
2. 둘째 열 개수 (int)
3. 세째 int[]은 열 포인터 (IA):
  - IA[0]=0 시작 값은 0으로, IA[i]=IA[i-1] + (i-1)열의 NNZ (NZ 개수, Num of NonZeroes)
4. 네째 int[]은 행 인덱스 (JA): 각 NZ의 행 인덱스
5. 마지막은 소수 (double)로 실제 값 리스트


2차원 dense vectors를 만들어 보자. 행의 갯수, 열의 갯수, 실제 값을 넣어주면 생성된다.

- 6은 행 갯수
- 4는 열 갯수
- 다음 수는 행렬을 해체하여 연속적인 수로 나열

In [20]:
from pyspark.mllib.linalg import Matrices

dm = Matrices.dense(6,4,[1, 2, 0, 0, 0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 5, 6, 7, 0, 0, 0, 0, 0, 0, 8])

In [21]:
dm.toArray() #2차원 배열로 변환됨

array([[1., 0., 0., 0.],
       [2., 3., 0., 0.],
       [0., 0., 5., 0.],
       [0., 4., 6., 0.],
       [0., 0., 7., 0.],
       [0., 0., 0., 8.]])

다음을 희소행렬로 변환해보자.

> [ 1.,  0.,  0.,  0.]  
[ 2.,  3.,  0.,  0.]  
[ 0.,  0.,  5.,  0.]  
[ 0.,  4.,  6.,  0.]  
[ 0.,  0.,  7.,  0.]  
[ 0.,  0.,  0.,  8.]

- 6은 행 갯수
- 4는 열 갯수


- 다음은 열포인터 [0, 2, 4, 7, 8]
  - 열로 세어서 요소의 개수 2, 2, 3, 1을 가지고 구성을 한다. 즉 열로 요소가 2, 2, 3, 1개 이다.
  - 0:IA[0]=0으로 시작, **2**:IA[1]=IA[0]+2, **4**:IA[2]=IA[1]+2 **7**:IA[3]=IA[2]+3, **8**:IA[4]=IA[3]+1
  
  
- 다음은 행인덱스 JA [0, 1, 1, 3, 2, 3, 4, 5]
  - 행으로 세어서 (단 컬럼순을 지켜서) 0 1 1 3 2 3 4 5 -- 열 순서대로 센다(0 1 1 2 3 3 4 5가 아니라)
  - 1:0행, 2:1행, 3:1행, 4:3행, 5:2행, 6:3행, 7:4행, 8:5행
  
  
- 마지막은 소수로 실제 값 리스트 [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [22]:
dm.toSparse()

SparseMatrix(6, 4, [0, 2, 4, 7, 8], [0, 1, 1, 3, 2, 3, 4, 5], [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], False)

In [23]:
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])

In [24]:
d=sm.toDense()

In [25]:
print (d)

DenseMatrix([[9., 0.],
             [0., 8.],
             [0., 6.]])


### S.4.3 분산 Matrix (Distributed Matrix)
- matrix가 나누어져서 존재

#### Row Matrix

In [26]:
p = [[1.0,2.0,3.0],[1.1,2.1,3.1],[1.2,2.2,3.3]]

In [27]:
my=spark.sparkContext.parallelize(p) #RDD 만들기

In [28]:
my.collect() #RDD 출력

[[1.0, 2.0, 3.0], [1.1, 2.1, 3.1], [1.2, 2.2, 3.3]]

In [29]:
from pyspark.mllib.linalg.distributed import RowMatrix

rm=RowMatrix(my)

In [30]:
print (type(rm))

<class 'pyspark.mllib.linalg.distributed.RowMatrix'>


In [31]:
rm.rows.collect()

[DenseVector([1.0, 2.0, 3.0]),
 DenseVector([1.1, 2.1, 3.1]),
 DenseVector([1.2, 2.2, 3.3])]

In [32]:
rm.numRows()

3

In [33]:
rm.numCols()

3

#### Indexed Row Matrix

In [34]:
from pyspark.mllib.linalg.distributed import IndexedRow

# RDD 생성
irRdd = spark.sparkContext.parallelize([
    IndexedRow(1, [3, 1, 2]), #2개의 요소로 되어있는 형식, 앞의 번호: 순서가 있는 data(시계열 data)일 때
    IndexedRow(2, [1, 3, 2]),
    IndexedRow(3, [5, 4, 3]),
    IndexedRow(4, [6, 7, 4]),
    IndexedRow(5, [8, 9, 2]),
])

In [35]:
from pyspark.mllib.linalg.distributed import IndexedRowMatrix

irm = IndexedRowMatrix(irRdd)

In [36]:
print(irm.numRows())
print(irm.numCols())

6
3


### S.4.4 Labeled Point
- 로컬벡터 - 레이블을 가지고있는 밀집or 희소행렬
- 레이블은 **double 형식**

#### label, features로 구성
분류 및 회귀분석에 사용되는 데이터 타잎이다. 'label'과 'features'로 구성된다.

구분|지도학습을 하기 위한 label과 features의 구성
:---:|:---:
label|supervised learning에서 '구분 값'으로 사용한다. 데이터타입은 'DoubleType'으로 설정되어야 한다.
features|sparse, dense 모두 사용할 수 있다.

In [37]:
from pyspark.mllib.regression import LabeledPoint

LabeledPoint(1.0, [1.0, 2.0, 3.0])

LabeledPoint(1.0, [1.0,2.0,3.0])

In [38]:
# sparse vectors(희소행렬)로 features 구성해보기

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

LabeledPoint(1992, Vectors.sparse(10, {0: 3.0, 1:5.5, 2: 10.0})) #개수, labled point

LabeledPoint(1992.0, (10,[0,1,2],[3.0,5.5,10.0]))

서로 다른 패키지의 데이터타잎 `mllib LabeledPoint`와 `ml Vectors`를 혼용하면, 형변환 오류가 발생한다. 이러한 오류는 패키지를 혼용하지 않으면 된다.

In [39]:
from pyspark.mllib.regression import LabeledPoint

LabeledPoint(1.0, dv1mllib)

LabeledPoint(1.0, [1.0,2.1,3.0])

In [40]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

LabeledPoint(1.0, Vectors.fromML(dv1ml))

LabeledPoint(1.0, [1.0,2.1,3.0])

### S.4.5 svm의 입력파일 형식
limsvm은 기계학습 모델인 svm을 위한 입력데이터 형식이다. 0은 label, 나머지는 index:value 쌍으로 구성한다.
>[label] [index1]:[value1] [index2]:[value2] ...  
[label] [index1]:[value1] [index2]:[value2] ...

- ex  
> 0 128:51 129:159 130:253 131:159 132:50 155:48 156:238 157:252 158:252 159:252 160:237 182:54 183:227 184:253 185:252 186:239 187:233 ...

#### DataFrame 읽기

In [41]:
import os
fsvm=os.path.join(os.environ["SPARK_HOME"],'data','mllib','sample_libsvm_data.txt')
dfsvm = spark.read.format("libsvm").load(fsvm) #별도의 형식이라 지정이 필요

In [42]:
type(dfsvm)

pyspark.sql.dataframe.DataFrame

In [43]:
dfsvm.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [44]:
dfsvm.take(1) #label에 features로 별도로 구성되어있음을 알 수 있다.

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0

#### MLUtils를 사용하여 RDD 읽기
또는 MLUtils.loadLibSVMFile로 읽을 수 있다.

In [45]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(spark.sparkContext, fsvm) #sparkContext라는 설정 해주어야한다.

In [46]:
label = data.map(lambda x: x.label)
features = data.map(lambda x: x.features) #map으로 label과 features 나누어 저장

In [47]:
label.take(5)

[0.0, 1.0, 1.0, 1.0, 1.0]

In [48]:
features.take(1)

[SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0, 356: 253.0, 357: 252.0

### S.4.6 TF

In [49]:
wikiRdd3 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))\
    .map(lambda line: line.split())

In [50]:
from pyspark.mllib.feature import HashingTF

hashingTF = HashingTF()
tf = hashingTF.transform(wikiRdd3) #rdd를 입력으로 해서 TF 생성
tf.collect() #row 벡터, 1048576: 전체 개수? 1026674: 1.0 -> 단어;빈도

[SparseVector(1048576, {1026674: 1.0}),
 SparseVector(1048576, {148618: 1.0, 183975: 1.0, 216207: 1.0, 261052: 1.0, 617454: 1.0, 696349: 1.0, 721336: 1.0, 816618: 1.0, 897662: 1.0}),
 SparseVector(1048576, {60386: 1.0, 177421: 1.0, 568609: 1.0, 569458: 1.0, 847171: 1.0, 850510: 1.0, 1040679: 1.0}),
 SparseVector(1048576, {261052: 4.0, 816618: 4.0}),
 SparseVector(1048576, {60386: 4.0, 594754: 4.0}),
 SparseVector(1048576, {21980: 1.0, 70882: 1.0, 274690: 1.0, 357784: 1.0, 549790: 1.0, 597434: 1.0, 804583: 1.0, 829803: 1.0, 935701: 1.0}),
 SparseVector(1048576, {154253: 1.0, 261052: 1.0, 438276: 1.0, 460085: 1.0, 585459: 1.0, 664288: 1.0, 816618: 1.0, 935701: 2.0, 948143: 1.0, 1017889: 1.0}),
 SparseVector(1048576, {270017: 1.0, 472985: 1.0, 511771: 1.0, 718483: 1.0, 820917: 1.0}),
 SparseVector(1048576, {34116: 1.0, 87407: 1.0, 276491: 1.0, 348943: 1.0, 482882: 1.0, 549350: 1.0, 721336: 1.0, 816618: 1.0, 1025622: 1.0}),
 SparseVector(1048576, {1769: 1.0, 151357: 1.0, 500659: 1.0, 54776

### S.4.7 TF-IDF
IDF는 전체에서 몇 개의 문서에 씌였는지를 반대로 계산한 값이다. 뒤 DataFrame를 사용하여 TF-IDF를 계산하면서 자세히 설명하기로 한다.

지도학습 훈련 data를 만들기 위해서 한다.

In [51]:
from pyspark.mllib.feature import HashingTF, IDF

idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [52]:
tfidf.collect()

[SparseVector(1048576, {1026674: 1.7047}),
 SparseVector(1048576, {148618: 1.7047, 183975: 1.7047, 216207: 1.7047, 261052: 1.0116, 617454: 1.7047, 696349: 1.7047, 721336: 1.2993, 816618: 0.7885, 897662: 1.7047}),
 SparseVector(1048576, {60386: 1.2993, 177421: 1.7047, 568609: 1.7047, 569458: 1.7047, 847171: 1.7047, 850510: 1.7047, 1040679: 1.7047}),
 SparseVector(1048576, {261052: 4.0464, 816618: 3.1538}),
 SparseVector(1048576, {60386: 5.1971, 594754: 6.819}),
 SparseVector(1048576, {21980: 1.7047, 70882: 1.7047, 274690: 1.7047, 357784: 1.7047, 549790: 1.7047, 597434: 1.7047, 804583: 1.7047, 829803: 1.7047, 935701: 1.2993}),
 SparseVector(1048576, {154253: 1.7047, 261052: 1.0116, 438276: 1.7047, 460085: 1.7047, 585459: 1.7047, 664288: 1.7047, 816618: 0.7885, 935701: 2.5986, 948143: 1.7047, 1017889: 1.7047}),
 SparseVector(1048576, {270017: 1.7047, 472985: 1.7047, 511771: 1.7047, 718483: 1.7047, 820917: 1.7047}),
 SparseVector(1048576, {34116: 1.7047, 87407: 1.7047, 276491: 1.7047, 3489

### S.4.8 StandardScaler
데이터를 표준화하려면 1) 평균과 표준편차를 계산하고, 2) 측정값에서 평균을 빼고, 표준편차로 나누어 주면 된다. 즉 zscore를 계산하는 것과 같다.

$$ z = \frac {\bar{x_n} - \mu} {\sigma / \sqrt{n}} $$

#### 데이터 생성

In [53]:
tRdd = spark.sparkContext\
    .textFile(os.path.join('data', 'ds_spark_heightweight.txt'))

#### 정규화 할 값만 추출
탭으로 분리한다.

In [54]:
tRdd.map(lambda x: x.split('\t')).take(1)

[['1', '65.78', '112.99']]

In [55]:
tRdd.map(lambda x: x.split('\t')).map(lambda x: [str(x[0]), float(x[1]), float(x[2])]).take(1)

[['1', 65.78, 112.99]]

In [56]:
#몇개의 명령어 붙임: pipeline
tRdd.map(lambda x: x.split('\t'))\
    .map(lambda x: [str(x[0]), float(x[1]), float(x[2])])\
    .take(1)

[['1', 65.78, 112.99]]

In [57]:
from pyspark.mllib.linalg import Vectors

_tRdd =tRdd\
    .map(lambda x: x.split('\t'))\
    .map(lambda x: [str(x[0]), float(x[1]), float(x[2])])\
    .map(lambda x: Vectors.dense([x[1], x[2]]))

In [58]:
from pyspark.mllib.linalg import Vectors

_tRdd =tRdd\
    .map(lambda x: x.split('\t'))\
    .map(lambda x: [str(x[0]), float(x[1]), float(x[2])])\
    .map(lambda x: [x[1], x[2]])

#### 표준화

In [59]:
from pyspark.mllib.feature import StandardScaler
scaler1 = StandardScaler().fit(_tRdd)
scaler2 = StandardScaler(withMean=True, withStd=True).fit(_tRdd) #평균, 표춘편차 적용하여 fit


In [60]:
scaler2.transform(_tRdd).take(5)

[DenseVector([-1.2458, -1.2299]),
 DenseVector([1.9011, 0.5934]),
 DenseVector([0.7388, 1.8767]),
 DenseVector([0.0919, 1.0473]),
 DenseVector([-0.1439, 1.1993])]

##

###

####